In [9]:
import numpy as np

from optint.data import synthetic_instance

import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
from rpy2.robjects.conversion import localconverter

print("rpy2 imported successfully!")

bidag = importr('BiDAG')
base = importr('base')

rpy2 imported successfully!


In [3]:
p = 5
seed = 42

# Create synthetic problem
problem = synthetic_instance(
    nnodes=p,
    DAG_type="random",
    sigma_square=0.4 * np.ones(p),
    a_size=p,
    a_target_nodes=None,
    std=False,
    seed=seed,
)

print(f"True DAG adjacency matrix:")
print(problem.DAG)
print(f"\nTrue B matrix:")
print(problem.B)

True DAG adjacency matrix:
[4][2][0][3|0][1|3]

True B matrix:
[[ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.         -0.53688887  0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.70873987  0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]]


In [4]:
# Generate observational data
n_obs = 50
X_obs = problem.sample(a=np.zeros((p, 1)), n=n_obs)

print(f"Observational data shape: {X_obs.shape}")  # Should be (p, n_obs)
print(f"First 3 samples:\n{X_obs[:, :3]}")

# Generate 3 intervention batches
n_per_batch = 20
batches = [X_obs]  # Start with observational
interventions = [np.zeros((p, 1))]  # a=0 for observational

for i in range(3):
    # Random intervention
    a = np.random.uniform(-1, 1, (p, 1))
    a = a / np.linalg.norm(a)
    
    batch = problem.sample(a, n_per_batch)
    batches.append(batch)
    interventions.append(a)
    
    print(f"\nIntervention {i+1}: a = {a.flatten()}")
    print(f"Batch shape: {batch.shape}")

print(f"\nTotal batches: {len(batches)}")

Observational data shape: (5, 50)
First 3 samples:
[[ 1.61876236  0.24933486  0.07729819]
 [-0.57346892 -0.18289798 -0.46155853]
 [-0.49036561  0.48897721 -0.50712046]
 [ 0.59853702  0.26896469 -0.81159822]
 [ 0.10436958 -0.54133698 -0.0250681 ]]

Intervention 1: a = [ 0.23712045  0.00590872  0.67162791  0.29897927 -0.63503253]
Batch shape: (5, 20)

Intervention 2: a = [ 0.10660719  0.50722503 -0.57473855 -0.2769506   0.56950122]
Batch shape: (5, 20)

Intervention 3: a = [-0.21976944 -0.06281145  0.41489099 -0.77676178 -0.41504543]
Batch shape: (5, 20)

Total batches: 4


In [5]:
def create_intervention_matrix(batches):
    """
    Convert list of batches to format expected by BiDAG.
    
    Args:
        batches: list of (p, n_i) arrays
        
    Returns:
        data: (total_samples, p) array
        Imat: (total_samples, n_conditions) binary array
    """
    n_conditions = len(batches)
    total_samples = sum(batch.shape[1] for batch in batches)
    p = batches[0].shape[0]
    
    # Pre-allocate
    data = np.zeros((total_samples, p))
    Imat = np.zeros((total_samples, n_conditions))
    
    # Fill in data and intervention indicators
    sample_idx = 0
    for cond_idx, batch in enumerate(batches):
        n_samples = batch.shape[1]
        
        # Transpose batch from (p, n) to (n, p) and insert
        data[sample_idx:sample_idx + n_samples, :] = batch.T
        
        # Mark which condition these samples came from
        Imat[sample_idx:sample_idx + n_samples, cond_idx] = 1
        
        sample_idx += n_samples
    
    return data, Imat


# Test it
data, Imat = create_intervention_matrix(batches)

print(f"Data shape: {data.shape}")  # Should be (total_samples, p)
print(f"Imat shape: {Imat.shape}")  # Should be (total_samples, 4)
print(f"\nIntervention matrix (first 10 rows):")
print(Imat[:10, :])
print(f"\nIntervention matrix (check each condition):")
for i in range(Imat.shape[1]):
    print(f"Condition {i}: {int(Imat[:, i].sum())} samples")

Data shape: (110, 5)
Imat shape: (110, 4)

Intervention matrix (first 10 rows):
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]

Intervention matrix (check each condition):
Condition 0: 50 samples
Condition 1: 20 samples
Condition 2: 20 samples
Condition 3: 20 samples


In [11]:
def compute_ibge_score(data, Imat, dag_adjacency, am=0.1):
    """
    Compute iBGe score for a given DAG.
    
    Args:
        data: (n_samples, p) numpy array
        Imat: (n_samples, n_conditions) binary numpy array  
        dag_adjacency: (p, p) binary adjacency matrix
        am: hyperparameter (default 0.1)
    
    Returns:
        score: log iBGe score
    """
    # Define R function to compute score (only once)
    ro.r('''
        compute_score_ibge <- function(data, Imat, dag, am) {
            library(BiDAG)
            
            # Create scoring object with iBGe
            scoreObj <- scoreparameters(
                scoretype = "usr",
                data = data,
                usrpar = list(pctesttype = "bge", Imat = Imat, am = am)
            )
            
            # Compute score for this DAG
            score <- DAGscore(scoreObj, dag)
            
            return(score)
        }
    ''')
    
    # Convert numpy arrays to R objects within conversion context
    with localconverter(ro.default_converter + numpy2ri.converter):
        r_data = ro.conversion.py2rpy(data)
        r_Imat = ro.conversion.py2rpy(Imat)
        r_dag = ro.conversion.py2rpy(dag_adjacency.astype(int))
    
    # Call the R function
    compute_fn = ro.r['compute_score_ibge']
    score = compute_fn(r_data, r_Imat, r_dag, am)[0]
    
    return score

In [16]:
# Use the correctly formatted DAG
true_dag = problem.DAG.to_amat()[0]  # Already extracted this above

print(f"True DAG adjacency:\n{true_dag}")
print(f"\nComputing iBGe score...")

# Compute iBGe score
score_true = compute_ibge_score(data, Imat, true_dag, am=0.1)

print(f"✓ iBGe score for TRUE DAG: {score_true:.2f}")

True DAG adjacency:
[[0 0 0 1 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 1 0 0 0]
 [0 0 0 0 0]]

Computing iBGe score...
✓ iBGe score for TRUE DAG: -60.94


In [17]:
# Create a random wrong DAG
np.random.seed(123)
wrong_dag = np.random.binomial(1, 0.3, size=(p, p))
# Make it acyclic by keeping only upper triangular
wrong_dag = np.triu(wrong_dag, k=1)

print(f"Wrong DAG adjacency:\n{wrong_dag}")
print(f"\nComputing iBGe score...")

score_wrong = compute_ibge_score(data, Imat, wrong_dag, am=0.1)

print(f"\n{'='*50}")
print(f"iBGe score for WRONG DAG: {score_wrong:.2f}")
print(f"iBGe score for TRUE DAG:  {score_true:.2f}")
print(f"{'='*50}")
print(f"Difference: {score_true - score_wrong:.2f}")
print(f"\n{'✓ TRUE DAG has HIGHER score!' if score_true > score_wrong else '✗ Something is wrong...'}")

Wrong DAG adjacency:
[[0 0 0 0 1]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]

Computing iBGe score...

iBGe score for WRONG DAG: -101.36
iBGe score for TRUE DAG:  -60.94
Difference: 40.41

✓ TRUE DAG has HIGHER score!


In [19]:
from optint.utils import learn_dag

# Learn DAG from the combined data using GES
all_data = np.hstack(batches)  # Concatenate all batches
learned_dag_ges_obj = learn_dag(all_data, p)

print(f"Learned DAG object: {learned_dag_ges_obj}")

# Extract adjacency matrix
learned_dag_ges = learned_dag_ges_obj.to_amat()[0]
print(f"\nLearned DAG adjacency matrix:")
print(learned_dag_ges)

# Score the learned DAG using iBGe
score_ges = compute_ibge_score(data, Imat, learned_dag_ges, am=0.1)

print(f"\n{'='*50}")
print(f"iBGe score for TRUE DAG:    {score_true:.2f}")
print(f"iBGe score for LEARNED DAG: {score_ges:.2f}")
print(f"iBGe score for WRONG DAG:   {score_wrong:.2f}")
print(f"{'='*50}")

# Check structural hamming distance
from optint.utils import compute_shd
shd = compute_shd(learned_dag_ges_obj, problem.DAG)  # Pass DAG objects, not arrays
print(f"\nStructural Hamming Distance (GES vs True): {shd}")

Learned DAG object: [4][2][1][0][3|0,1]

Learned DAG adjacency matrix:
[[0 0 0 1 0]
 [0 0 0 1 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]

iBGe score for TRUE DAG:    -60.94
iBGe score for LEARNED DAG: -73.42
iBGe score for WRONG DAG:   -101.36

Structural Hamming Distance (GES vs True): 2


In [20]:
def test_bayesian_active_ibge(problem, G_init, opts, K=1, am=0.1):
    """
    Active learning with DAG bootstrap and iBGe scoring.
    
    Args:
        problem: synthetic problem instance
        G_init: initial DAG guess
        opts: options (T, W, n, etc.)
        K: number of bootstrap samples (K=1 for single DAG)
        am: iBGe hyperparameter
    
    Returns:
        A: list of interventions
        Prob: list of posterior distributions
        SHD: list of structural hamming distances
    """
    from optint.model import linearSCM
    from optint.acquisition import civ_acq
    from optint.utils import learn_dag, compute_shd
    
    A = []
    Prob = []
    SHD = []
    
    all_batches = []
    all_interventions = []
    
    # Start with initial DAG
    current_dag = G_init.to_amat()[0]  # Convert to numpy array
    model = linearSCM(G_init, {'pot_vec': 0, 'info_mat': 1})
    
    print(f"Starting active learning with K={K} bootstrap samples")
    print(f"Initial SHD: {compute_shd(G_init, problem.DAG)}")
    
    # Warm-up with random interventions
    for w in range(opts.W):
        a = np.random.uniform(-1, 1, (problem.nnodes, 1))
        a = a / np.linalg.norm(a)
        
        batch = problem.sample(a, opts.n)
        model.update_posterior(a, batch)
        
        all_batches.append(batch)
        all_interventions.append(a)
        
        A.append(a)
        Prob.append(model.prob_padded())
        SHD.append(compute_shd(G_init, problem.DAG))
    
    print(f"Warm-up complete: {opts.W} random interventions")
    
    # Active learning loop
    for i in range(opts.T - opts.W):
        # Get posterior for acquisition function
        prob_pad = model.prob_padded()
        mean = np.array(prob_pad['mean'])
        var = np.array(prob_pad['var'])
        
        sigma_square = problem.sigma_square if opts.known_noise else np.zeros(problem.sigma_square.shape)
        
        # Compute acquisition function
        acquisition = civ_acq(sigma_square, mean, var, problem.mu_target, opts.n, opts.measure)
        
        # Optimize acquisition (same as before)
        try:
            a_jitter = A[-1].reshape(-1) * np.random.uniform(0.8, 1.2, (problem.nnodes,))
            x01 = np.maximum(np.minimum(a_jitter, 1), -1)
            
            x02 = problem.mu_target - np.matmul(mean, problem.mu_target)
            x02 = x02.flatten()
            x02 = x02 / np.linalg.norm(x02)
            
            a1 = acquisition.optimize(x0=x01)
            a2 = acquisition.optimize(x0=x02)
            
            if a1 is not None and a2 is not None:
                a1 = a1.reshape(-1, 1)
                a2 = a2.reshape(-1, 1)
                a = (a1, a2)[acquisition.evaluate(a1) > acquisition.evaluate(a2)]
            elif a1 is not None:
                a = a1.reshape(-1, 1)
            elif a2 is not None:
                a = a2.reshape(-1, 1)
            else:
                print("Both initializations failed...")
                a = np.random.uniform(-1, 1, problem.nnodes).reshape(-1, 1)
                a = a / np.linalg.norm(a)
        except:
            x0 = np.random.uniform(-1, 1, problem.nnodes)
            x0 = x0 / np.linalg.norm(x0)
            a = acquisition.optimize(x0=x0)
            if a is None:
                a = np.random.uniform(-1, 1, problem.nnodes).reshape(-1, 1)
                a = a / np.linalg.norm(a)
            else:
                a = a.reshape(-1, 1)
        
        # Sample from true system
        batch = problem.sample(a, opts.n)
        all_batches.append(batch)
        all_interventions.append(a)
        
        # === DAG LEARNING WITH iBGe ===
        # Format data for iBGe
        data_for_scoring, Imat = create_intervention_matrix(all_batches)
        
        if K == 1:
            # Single DAG: just learn with GES and score with iBGe
            all_data = np.hstack(all_batches)
            learned_dag_obj = learn_dag(all_data, problem.nnodes)
            learned_dag = learned_dag_obj.to_amat()[0]
            
            score = compute_ibge_score(data_for_scoring, Imat, learned_dag, am=am)
            
            current_dag = learned_dag
            current_dag_obj = learned_dag_obj
            
        else:
            # DAG Bootstrap: sample K DAGs
            dags = []
            dag_objs = []
            scores = []
            
            for k in range(K):
                # Bootstrap sample the data
                n_samples = sum(batch.shape[1] for batch in all_batches)
                bootstrap_indices = np.random.choice(n_samples, size=n_samples, replace=True)
                
                # Reconstruct batches from bootstrap indices
                # (This is simplified - you might want to bootstrap per-batch)
                all_data = np.hstack(all_batches)
                bootstrap_data = all_data[:, bootstrap_indices]
                
                # Learn DAG on bootstrap sample
                dag_obj_k = learn_dag(bootstrap_data, problem.nnodes)
                dag_k = dag_obj_k.to_amat()[0]
                
                # Score on FULL dataset (not bootstrap)
                score_k = compute_ibge_score(data_for_scoring, Imat, dag_k, am=am)
                
                dags.append(dag_k)
                dag_objs.append(dag_obj_k)
                scores.append(score_k)
            
            # Compute importance weights (unnormalized posterior)
            scores = np.array(scores)
            # Convert log scores to probabilities (numerically stable)
            max_score = np.max(scores)
            exp_scores = np.exp(scores - max_score)
            weights = exp_scores / np.sum(exp_scores)
            
            # Use MAP DAG for now (highest scoring)
            best_idx = np.argmax(scores)
            current_dag = dags[best_idx]
            current_dag_obj = dag_objs[best_idx]
            
            print(f"Step {i+1}: K={K} DAGs, iBGe scores: [{scores.min():.1f}, {scores.max():.1f}], weights: {weights}")
        
        # Update model with new DAG
        model = linearSCM(current_dag_obj, {'pot_vec': 0, 'info_mat': 1})
        
        # Re-update posterior with all historical data
        for batch_k, a_k in zip(all_batches, all_interventions):
            model.update_posterior(a_k, batch_k)
        
        A.append(a)
        Prob.append(model.prob_padded())
        SHD.append(compute_shd(current_dag_obj, problem.DAG))
        
        if (i + 1) % 10 == 0:
            print(f"Step {i+1}/{opts.T-opts.W}: SHD = {SHD[-1]}")
    
    return A, Prob, SHD

In [21]:
# Create a fresh problem for testing
p = 10
seed = 100

problem_test = synthetic_instance(
    nnodes=p,
    DAG_type="random",
    sigma_square=0.4 * np.ones(p),
    a_size=p,
    a_target_nodes=None,
    std=False,
    seed=seed,
)

# Initial observational data
X_obs = problem_test.sample(a=np.zeros((p, 1)), n=100)
G_init = learn_dag(X_obs, p)

print(f"Initial DAG SHD: {compute_shd(G_init, problem_test.DAG)}")

# Set up options
class Opts:
    pass

opts = Opts()
opts.T = 20  # Total steps
opts.W = 3   # Warm-up steps
opts.n = 10  # Samples per intervention
opts.known_noise = True
opts.measure = 'unif'

# Run with K=1
print("\n" + "="*60)
print("TESTING WITH K=1 (single DAG)")
print("="*60)

A, Prob, SHD = test_bayesian_active_ibge(problem_test, G_init, opts, K=1)

print(f"\nFinal SHD: {SHD[-1]}")
print(f"SHD trajectory: {SHD[::5]}")  # Every 5 steps

Initial DAG SHD: 2

TESTING WITH K=1 (single DAG)
Starting active learning with K=1 bootstrap samples
Initial SHD: 2
Warm-up complete: 3 random interventions
Step 10/17: SHD = 2

Final SHD: 2
SHD trajectory: [2, 8, 2, 5]
